# Intersection Analysis and Tracing
Adapted from curli intersection analysis by Sebastian (July 07, 2022)

#### Definition:
Given a list of source nodes and a list of target nodes, find the potential most important nodes from sources to targets. 

#### Approaches:
1. Run personalized pagerank using source nodes. The pagerank value is considered to be the probability that the node is influenced by the source nodes. 
2. Run personalized reverse pagerank using target nodes. The reverse pagerank value is considered to be the probability that the node influences the target nodes.
3. Use a probability formula to get a intersection pagerank that represents the possibility that the node being influenced by the source nodes and also influences the target nodes.

<img src="img/intersection.png" width="400" height="200" />


### Steps to run Intersection Analysis
1. Connect to arango database
2. Find input nodes (source nodes and target nodes) in arango database
3. Load the whole network graph from arango to memory and create a networkx graph. NetworkX is a python network library.
4. Perform intersection analysis
    - Run personalized pagerank algorithm using source nodes to get pagerank values for each nodes 
that the source nodes can reach (forward direction). Those are the nodes influenced by the source nodes
    - Run personalized reverse pagerank using target nodes to get the reverce pagerank values for each 
nodes that can reach the target nodes
    - Calculate the intersection pageranks based on source pageranks and target reverse pageranks
    - Export the pagerank values into __excel file__
5. The user analyzes the pagerank values (sorting, filtering etc), and select the rows that are interesting
6. Create intersection traces for selected nodes

In [1]:
# install lifelike_gds package if not already installed (e.g. running in Google Colab)
import importlib

if importlib.util.find_spec('lifelike_gds') is None:
  !pip install git+https://github.com/SBRG/GDS-Public

# provide the path to the notebook folder in the github repository in case the notebook is run in Google Colab
github_path = 'SBRG/GDS-Public/main/notebooks/reactome'

In [2]:
import os
import pandas as pd
import warnings
from pathlib import PurePosixPath

In [3]:
from lifelike_gds.arango_network.reactome import *
from lifelike_gds.arango_network.radiate_trace import RadiateTrace

/Users/dommas/.local/share/virtualenvs/GDS-Public-uKBS7cUg/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
warnings.filterwarnings('ignore')

In [5]:
input_dir = PurePosixPath('input')
output_dir = PurePosixPath('output')
os.makedirs(output_dir, 0o777, True)
# gds database name
arango_dbname = 'reactome'
# gds database version, free text, that can be used to describe the graph
db_version = 'reactome-arango-test-1'

## 1. Connect to arango database.
If use BioCyc databases (e.g. EcoCyc, HumanCyc), use Class BioCycDB.  
If use Reactome database, use Class ReactomeDB. 

In [6]:
# set database uri, username and password. 
# dbname is the arango database name for the running arango instance. The default database name is 'arango'
database = ReactomeDB(dbname=arango_dbname)

## 2a. Define functions to get source/target nodes
read input file with ids (stId or dbId)
read input file with reference ids (gene_id or chebi_id)

In [7]:
"""
read csv file to get list of reactome nodes

Parameters
----------
csv_filename:  the input file
id_name: the property name in reactome db, e.g. stId, dbId
id_column: the column name for the id property
""" 
def get_nodes_by_identity_from_file(csv_filename, id_name, id_column, sep=','):
    csv_file_path = input_dir / csv_filename
    if os.path.isfile(csv_file_path):
      csv_file_ref = csv_file_path
    else:
      # if does not exist localy, pull from github
      csv_file_ref = f'https://raw.githubusercontent.com/{github_path}/{csv_file_path}'
        
    df = pd.read_csv(csv_file_ref, sep=sep, dtype='str')
    ids = [n for n in df[id_column]]
    nodes = database.get_nodes_by_attr(ids, id_name)
    print('file_rows:', len(df), ', nodes matched:', len(nodes))
    return nodes


def get_chemical_nodes_by_chebi(csv_filename, chebi_id_column, sep=','):
    csv_file_path = input_dir / csv_filename
    if os.path.isfile(csv_file_path):
      csv_file_ref = csv_file_path
    else:
      # if does not exist localy, pull from github
      csv_file_ref = f'https://raw.githubusercontent.com/{github_path}/{csv_file_path}'
        
    df = pd.read_csv(csv_file_ref, sep=sep, dtype='str')
    ids = [n for n in df[chebi_id_column]]
    nodes = database.get_entity_nodes_by_chebi_ids(ids)
    print('file_rows:', len(df), ', nodes matched:', len(nodes))
    return nodes

def get_protein_nodes_by_gene_id(csv_filename, gene_id_column, sep=','):
    csv_file_path = input_dir / csv_filename
    if os.path.isfile(csv_file_path):
      csv_file_ref = csv_file_path
    else:
      # if does not exist localy, pull from github
      csv_file_ref = f'https://raw.githubusercontent.com/{github_path}/{csv_file_path}'
        
    df = pd.read_csv(csv_file_ref, sep=sep, dtype='str')
    ids = [n for n in df[gene_id_column]]
    nodes = database.get_entity_nodes_by_gene_ids(ids)
    print('file_rows:', len(df), ', nodes matched:', len(nodes))
    return nodes

def get_reference_nodes_by_chebi(csv_filename, chebi_id_column, sep=','):
    csv_file_path = input_dir / csv_filename
    if os.path.isfile(csv_file_path):
      csv_file_ref = csv_file_path
    else:
      # if does not exist localy, pull from github
      csv_file_ref = f'https://raw.githubusercontent.com/{github_path}/{csv_file_path}'
        
    df = pd.read_csv(csv_file_ref, sep=sep, dtype='str')
    ids = [n for n in df[chebi_id_column]]
    nodes = database.get_reference_nodes_by_chebi_ids(ids)
    print('file_rows:', len(df), ', nodes matched:', len(nodes))
    return nodes

def get_reference_nodes_by_gene_id(csv_filename, gene_id_column, sep=','):
    csv_file_path = input_dir / csv_filename
    if os.path.isfile(csv_file_path):
      csv_file_ref = csv_file_path
    else:
      # if does not exist localy, pull from github
      csv_file_ref = f'https://raw.githubusercontent.com/{github_path}/{csv_file_path}'
        
    df = pd.read_csv(csv_file_ref, sep=sep, dtype='str')
    ids = [n for n in df[gene_id_column]]
    nodes = database.get_reference_nodes_by_gene_ids(ids)
    print('file_rows:', len(df), ', nodes matched:', len(nodes))
    return nodes

## 2b. Find input nodes (source and target nodes) in arango database

#### Read updown genes as sources, and metabolites as targets

In [8]:
# sources: updown genes
updown_genes_file = 'updown.entrez'

In [9]:
updown_genes_file_path = input_dir / updown_genes_file
if os.path.isfile(updown_genes_file_path):
  updown_genes_file_ref = updown_genes_file_path
else:
  # if does not exist localy, pull from github
  updown_genes_file_ref = f'https://raw.githubusercontent.com/{github_path}/{updown_genes_file_path}'

In [10]:
#df1 = pd.read_csv(updown_genes_file_ref))
#updown_genes = [n for n in df1['gene_id']]
updown_nodes = get_protein_nodes_by_gene_id(updown_genes_file, gene_id_column='gene_id')
#print(f"updown genes: {len(updown_genes)}, nodes: {len(updown_nodes)}")


# targets: metabolites
metabs_file = 'metabolite.txt'

INFO: 92 gene_ids, matched to 119 nodes


file_rows: 92 , nodes matched: 119


In [11]:
metabs_file_path = input_dir / metabs_file
if os.path.isfile(metabs_file_path):
  metabs_file_ref = metabs_file_path
else:
  # if does not exist localy, pull from github
  metabs_file_ref = f'https://raw.githubusercontent.com/{github_path}/{metabs_file_path}'

In [12]:
#df3 = pd.read_csv(metabs_file_ref)
#metabs = [n for n in df3['chebi']]

######to be changed to reactome mapping function (get from radiate script)
metabs_nodes = get_chemical_nodes_by_chebi(metabs_file, chebi_id_column='chebi')
#print(f"metabolites: {len(metabs)}, nodes: {len(metabs_nodes)}")

INFO: 23 chebi_ids, matched to 13 nodes


file_rows: 23 , nodes matched: 13


## 3. Load the whole network graph from arango to memory and create a networkx graph

Create a RadiateTrace instance.  
RadiateTrace is a subclass of TraceGraphNx.  TraceGraphNx has a property __graph__, that is a networkx graph. After the graph is created by using data from arango graph database, all the algorithms and traces can be run using the python networkx library.

In [13]:
tracegraph = RadiateTrace(Reactome(database))
# set up output directory where the excel and graph files will write to
tracegraph.datadir = output_dir
# initiate tracegraph by loading graph data from arango
tracegraph.init_default_graph()

INFO: load reactome graph
INFO: MultiDirectedGraph with 71225 nodes and 112575 edges


## 4. Perform intersection analysis
Run intersection pagerank analysis and export values into excel file.

The pagerank analysis is performed using networkx graph that contains a set of nodes and set of edges. 

#### Set node sets for sources and targets.  
A node set is a list of node ids with a name and description.

We set two source node sets and one target node set.  Then we will perform intersection analysis from each source node set to the target node set

In [14]:
# Set source and target node sets
SOURCE_SET = 'updown_genes'
TARGET_SET = 'metabolites'

tracegraph.set_node_set_from_arango_nodes(updown_nodes, SOURCE_SET, 'updown_genes')
tracegraph.set_node_set_from_arango_nodes(metabs_nodes, TARGET_SET, 'metabolites')

#### Call export_intersection_pageranks
The method export_intersection_pageranks() performs the following steps
1. Run personalized pagerank using source nodes
2. Run personalized reverse pagerank using target nodes
3. Calculate intersection pagerank based on source pagerank and target reverse pagerank
4. Write values into excel file

In [15]:
# keep a clean copy of graph
tracegraph.graph = tracegraph.orig_graph.copy()

filename = f"Intersection_analysis_for_{SOURCE_SET}_and_{TARGET_SET}.xlsx"
tracegraph.export_intersection_pageranks(filename, SOURCE_SET, TARGET_SET, num_nodes=3000)

INFO: set pagerank and num reach for updown_genes
INFO: set pagerank and num reach for metabolites


export intersection pagerank to file  output/Intersection_analysis_for_updown_genes_and_metabolites.xlsx


## 5. Analyze the pagerank output file (excel), and select interesting rows for further analysis

#### Suggestion:   
Add a column 'select' for selecting top pagerank nodes, and set any selected rows to 1, then save the file 

## 6. Create intersection traces for the selected rows

#### Read manually selected top ranked nodes from the previous generated pagerank excel file
We will read the column 'select' to get the selected rows (for intersection pagerank selection)

In [16]:
intersect_pagerank_select_file_path = input_dir / f"Intersection_analysis_for_{SOURCE_SET}_and_{TARGET_SET}_select.xlsx"
if os.path.isfile(intersect_pagerank_select_file_path):
  intersect_pagerank_select_file_ref = intersect_pagerank_select_file_path
else:
  # if does not exist localy, pull from github
  intersect_pagerank_select_file_ref = f'https://raw.githubusercontent.com/{github_path}/{intersect_pagerank_select_file_path}'

In [17]:
#adapted by Sebastian to use stId-->double-check with Robin
df = pd.read_excel(intersect_pagerank_select_file_ref, usecols=['stId', 'select'])
df = df[df['select']==1]
df

HTTPError: HTTP Error 404: Not Found

In [ ]:
#adapted by Sebastian to use stId-->double-check with Robin
#also double-check if get_nodes_by_attr function is ok to use here
selected_stIds = [id for id in df['stId']]
print(selected_stIds)
selected_nodes = database.get_nodes_by_attr(selected_stIds, 'stId')
print('No. of selected nodes: ', len(selected_nodes))

#### Run pageranks again using a clean copy of the original graph

In [ ]:
tracegraph.graph = tracegraph.orig_graph.copy()

pr = 'pagerank'
rev_pr = 'rev_pagerank'
tracegraph.set_pagerank(SOURCE_SET, pagerank_prop=pr)
tracegraph.set_pagerank(TARGET_SET, pagerank_prop=rev_pr, reverse=True)

#### Add traces from sources to the intersection node, and intersection node to targets
We will add traces from source nodes to each selected intersection nodes, and traces from each selected intersection nodes to targets

In [ ]:
# write traces in one file
tracegraph.add_traces_from_sources_to_each_selected_nodes(selected_nodes, SOURCE_SET, weighted_prop=pr)
tracegraph.add_traces_from_each_selected_nodes_to_targets(selected_nodes, TARGET_SET, weighted_prop=rev_pr)

In [ ]:
tracegraph.write_to_sankey_file(f"Intersection_traces_from_{SOURCE_SET}_to_{TARGET_SET}.graph")